In [1]:
'''
Licensed Materials - Property of IBM
IBM Maximo APM - Predictive Maintenance Insights On-Premises
IBM Maximo APM - Predictive Maintenance Insights SaaS 
IBM Maximo Application Suite 
© Copyright IBM Corp. 2019, 2020, 2021 All Rights Reserved.
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.
'''

'\nLicensed Materials - Property of IBM\nIBM Maximo APM - Predictive Maintenance Insights On-Premises\nIBM Maximo APM - Predictive Maintenance Insights SaaS \nIBM Maximo Application Suite \n© Copyright IBM Corp. 2019, 2020, 2021 All Rights Reserved.\nUS Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.\n'

# Maximo Predict - Predicted Failure Date Model Template using Smart Regression

1. [Introduction](#introduction)
2. [Install Maximo Predict SDK](#install-maximo-apm-pmi-sdk)
3. [Setup the Model Training Pipeline](#setup-model-training-pipline)
4. [Train the Model Instance](#train-model-instance)
5. [Register the Trained Model Instance](#register-trained-model-instance)
6. [How to overide the training stages](# How to overide the training stages)

<a id='introduction'></a>
## Introduction

###### Use Case Description

This notebook deals with computing time to failure for a system like a process or a unit task or an asset like a device. 

This notebook uses the Smart Regression package of SROM supporting various algorithms to compute the time to failure. 
This model provides following estimators for analysis, including:
            * ("linearregression", LinearRegression()), 
            * ("ridge", Ridge()), 
            * ("lasso", Lasso()),
            * ("elasticnet", ElasticNet()),
            * ("lassolars", LassoLars()),
            * ("orthogonalmatchingpursuit", OrthogonalMatchingPursuit()),
            * ("bayesianridge", BayesianRidge()),
            * ("sgdregressor", SGDRegressor()),
            * ("passiveaggressiveregressor", PassiveAggressiveRegressor()),
            * ("kneighborsregressor", KNeighborsRegressor()),
            * ("decisiontreeregressor", DecisionTreeRegressor()),
            * ("mlpregressor", MLPRegressor()),
            * ("gradientboostingregressor", GradientBoostingRegressor()),
            * ("adaboostregressor", AdaBoostRegressor()),
            * ("baggingregressor", BaggingRegressor()),
            * ("randomforestregressor", RandomForestRegressor()),
            * ("extratreesregressor", ExtraTreesRegressor()),
            * ("plsregression", PLSRegression()),
            * ("gaussianprocessregressor", GaussianProcessRegressor()),
            * ("isotonicregression", IsotonicRegression()),
            * ("kernelridge", KernelRidge()),
            * ("theilsenregressor", TheilSenRegressor()),
            * ("ransacregressor", RANSACRegressor()),
            * ("huberregressor", HuberRegressor()),
            * ("partitionregressor", PartitionRegressor()),
            * ("xgbregressor", XGBRegressor()),
            

The notebook shows the model building, training, and registration to PMI.

**Input Data**
The raw data for time-to-failure model is from Maximo and IOT platform. First, it gets the asset failure history and the assets' sensor data (the *_episode). Then the raw data will be merged into the format with the columns below -

+ id (that indicates the process or the system or an asset)
+ event_timestamp (the timestamp of data points)
+ failuredate (the asset has failure on this date, which is coming from Maximo failure history data)
+ installdate (the installation date of the asset, which is coming from Maximo asset meta data)
+ *_episode (related sensor data to the failure event)

| id  | event_timestamp | faildate |  c_episode | t_episode | f_episode | p_episode| installdate | 
| ------------- |-------------|-------------|-------|------------|------------|------------|------------|
| SAMPLE_ASSET101-__-BEDFORD | 2018-07-02 | 2018-07-02 | 1 | 7  | 2 | 3| 2017-10-30T00:00:00 |
| SAMPLE_ASSET101-__-BEDFORD | 2019-03-18 | NaT | 1 | 5  | 2 | 5| 2017-10-30T00:00:00 |
| SAMPLE_ASSET102-__-BEDFORD | 2019-03-21 | 2019-03-21 | 2 | 8  | 4 | 3| 2017-10-29T00:00:00 |

In the data set above, each row corresponds to a device event. The data contains failure information for the devices. The failuredate means the asset has failure on this date, otherwise it is NaT. The model will calculate the durations between different failure dates for each asset. The remaining columns are covariates which are potentially correlated with failures of devices.

**Output**

Predicted time to failure in days for a given failure_mode, prediction_interval low bound and high bound for the same failure_mode, and   predicted_failure_date for the same failure mode.

Example:
     	predicted_time_to_failure_pumps_leak 	prediction_interval_pumps_leak_low 	prediction_interval_pumps_leak_high 	predicted_failure_date_pumps_leak
       	137.008 	                             72.9724 	                           214.6860 	                         2018-08-18 00:11:31.200

<a id="install-maximo-apm-pmi-sdk"></a>
## Install Maximo Predict SDK

You will need follow 3 credentials to run the notebook. You can obtain them if you are admin on Maximo Predict UI.
             APM_ID: Application Administration -> System Properties -> Filter -> Search PMIId -> Current Value (eg. b95ed774)
             APM_API_BASEURL: Application Administration -> Integration -> End Points--> Searh for predict -> click search result PREDICTAPI -> URL (e.g https://predict-api.mas-pmidev1-predict.svc, note you just need first part of the url)
             APM_API_KEY: Application Administration -> Go To Administration -> Copy key from user card (e.g. 6805t46gn3tef37pu0picpg9vcq3hsmamm1enc), or Add API key if not exist.

In [2]:
# This cell loads the json mediated integration between related notebooks. For this group the json is called 'multi_execution.json'
#
# This section adds the current notebook to the json and warns if precursors do not have outputs registered indicating they have not been run
# 

import json
from project_lib import Project
project = Project.access()
store = json.load(project.get_file("fast_execution.json"))
store
notebook_name = "PMI - Predicted Failure Date-Smart Regression"
notebook_precursors = ["FastStartLoader-Predict"]
store[notebook_name] = {"precursors":notebook_precursors, "outputs":{}}
store_json = json.dumps(store, indent =4)
project.save_data("fast_execution.json", store_json, overwrite=True)
print("Notebook added to json")
for precursor in notebook_precursors:
    for nb in store:
        if precursor == nb:
            #Found precursor. Check if it has output(meaning it has been run)
            if store[precursor]["outputs"] == {}:
                #Precursor has not been run. Must run before notebook can be run
                raise ValueError(precursor + " must run before this notebook.")
                
# This loads the variables generated by the precursor notebooks needed to run the current notebook.
                
device_type=store["FastStartLoader-Predict"]["outputs"]["device_type"]
print('device_type='+device_type)
asset_group_id = store["FastStartLoader-Predict"]["outputs"]["asset_group_id"]
print('asset_group_id='+asset_group_id)

Notebook added to json
device_type=Pump_DIST2_ST
asset_group_id=1628


In [3]:
!pip uninstall -y pmlib
!pip uninstall -y srom

Found existing installation: pmlib 8.4.0
Uninstalling pmlib-8.4.0:
  Successfully uninstalled pmlib-8.4.0
Found existing installation: srom 1.3.2.1.0
Uninstalling srom-1.3.2.1.0:
  Successfully uninstalled srom-1.3.2.1.0


In [5]:
import json
import os
# Opening JSON file
f = open('/project_data/data_asset/Predict_Envs.json',)
data = json.load(f)
#print(data)
f.close()
os.environ['APM_ID'] = data['APM_ID']
os.environ['APM_API_BASEURL'] = data['APM_API_BASEURL']
os.environ['APM_API_KEY'] = data['APM_API_KEY']


In [6]:
import os
os.environ['TRUST_PREDICT']= os.getenv('APM_API_BASEURL')[8:]
print(os.getenv('TRUST_PREDICT'))

ivt09rel86-predict-api.mas-ivt09rel86-predict.svc


Then, install Maximo Predict SDK with `pip`.

In [7]:

pip install --trusted-host ${TRUST_PREDICT}  -U ${APM_API_BASEURL}/ibm/pmi/service/rest/ds/${APM_ID}/${APM_API_KEY}/lib/download?filename=pmlib


     |████████████████████████████████| 129.2 MB 34.8 MB/s eta 0:00:01 57.6 MB/s eta 0:00:03     |████▎                           | 17.4 MB 57.6 MB/s eta 0:00:02  | 65.6 MB 57.6 MB/s eta 0:00:02
Processing //tmp/wsuser/pip-req-build-lfyax3jr/dslib/srom-1.3.2.1.0-cp37-cp37m-linux_x86_64.whl


  Created wheel for pmlib: filename=pmlib-8.4.0-py3-none-any.whl size=922960 sha256=acfd101aa69ef12bb2ed485a8c114c10bfb754838a29cc0ab61f5af5abdbe5e6
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-m2nlui3g/wheels/0f/28/95/1a8720989bdaf4b6460f899b741f7b1357998f810f72c5e106
Successfully built pmlib
ERROR: srom 1.3.2.1.0 has requirement dill==0.3.2, but you'll have dill 0.3.0 which is incompatible.
ERROR: srom 1.3.2.1.0 has requirement networkx>=2.5, but you'll have networkx 2.3 which is incompatible.
ERROR: srom 1.3.2.1.0 has requirement tsfresh==0.17.0, but you'll have tsfresh 0.10.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import srom
srom.__version__

'1.3.2'

In [9]:
# sklearn-deap=='0.2.3' to match scikit-learn==0.23.1 for October 2021 release
!pip install sklearn-deap=='0.2.3'

In [10]:
#pyspark version needs to be 3.0.1 for the srom version being used.
import pkg_resources

dists = [d for d in pkg_resources.working_set]

if not('pyspark' in dists):
   print ("pyspark does not exist. Installing ..." )
   !pip install pyspark==3.0.1
else:
   print ("pyspark already exists. Not installing")

pyspark does not exist. Installing ...


In [11]:
#use the following flags to suppress the pointless pyspark logs that fill up space
import logging
logger_pyspark = logging.getLogger('pyspark').setLevel(logging.ERROR)
logger_py4j = logging.getLogger('py4j').setLevel(logging.ERROR)
logger_py4java = logging.getLogger('py4j.java_gateway').setLevel(logging.ERROR)

<a id="setup-model-training-pipline"></a>
## Setup the Model Training Pipeline

Before you can start working on the model training pipeline, you have to setup an asset group and asset-sensor relationshp properly in Maximo. See IBM Maximo APM - Predictive MaintenanceInsights SaaS User Guide for details.

Required model pipeline configuration:

* Asset group ID: The unit of model processing is an asset group. Asset groups are managed on Maximo APM UI. You need to get the ID of the asset group to be analyzed by this model.
* Asset failure history and installation date as the label: This model requires asset installation date (Asset attribute **```installdate```** in Maximo) and asset failure history (Asset Workorder attribute **```faildate```** in Maximo) to extract the latel for training.
* Sensor data as features: This model also accepts one or more features from either asset data or IOT data. **Note that these features must be of type Integer or Floating-Point Number.** Features are specifed simply by attribute names prefixed by type (with separator colon). For asset attribute, the prefix is an empty string. For IOT data attribute, the prefix is the device type registered on Watson IOT Platform.
* Prediction output names: This model generates one output, the predicted failure date. Give it a name (containing only alphanumeric, dash, and underline).

Now you can setup a training pipeine based on this model template, with your own data, to train a model instance.

In the model_pipeline, please follow the below configuration:
* 'features': required
* 'predictions': ['predicted_time_to_failure','predicted_failure_date']   --- required
* 'failure_mode':  required. Format: CLASS/PROBLEMCODE or CLASS/PROBLEMCODE/CAUSE or CLASS/PROBLEMCODE/CAUSE/REMEDY. The CLASS, PROBLEMCODE, CAUSE, and REMEDY are defined in IBM MAXIMO MANAGE. The CLASS and PROBLEMCODE are required to be set up in MAXIMO MANAGE.  Example: PUMPS/STOPPED, PUMPS/STOPPED/MOTRFAIL, PUMPS/STOPPED/MOTRFAIL/REPLACE.      
* 'aggregate_methods':  required, can be min, max, median,std
* 'aggregate_window_size':         optional, If it is present, then it must be positive integer. Example: 3d
* 'aggregate_type_for_prediction_interval': 'mean'   ---optional. If it is not present, median is used.
* 'smart_regression': required. It must be "True".


In [12]:
import pmlib

You can get the asset_group_id from the Maximo Health->Predict grouping page. You need to set asset_group_id.

In [13]:
#desired_features = ['load', 'speed', 'acc']
#device_type='ROBOT_TYPE'
#device_type_qualified_features = [device_type+':'+col for col in desired_features]
#print(device_type_qualified_features)
#device_type='Pump_Afm'
device_type=store["FastStartLoader-Predict"]["outputs"]["device_type"]
iot_type=device_type
desired_features = ['VELOCITYX','VELOCITYY','VELOCITYZ']
device_type_qualified_features = [device_type+':'+col for col in desired_features]
print(device_type_qualified_features)

['Pump_DIST2_ST:VELOCITYX', 'Pump_DIST2_ST:VELOCITYY', 'Pump_DIST2_ST:VELOCITYZ']


Call pmlib.api.get_asset_failure_history to get possible values of failure_mode.

In [14]:
# Replace OEMDEVICE00 with your own assetNum and siteId with your siteId.
# assets = [{'assetNum': 'OEMDEVICE00'+str(n), 'siteId': 'DIST1'} for n in range(2,10,2)]
# assets = [{'assetNum': 'ST_1393137', 'siteId': 'DIST2'},{'assetNum': 'ST_1393138', 'siteId': 'DIST2'},{'assetNum': 'ST_1400501', 'siteId': 'DIST2'}]
# # assets = [{'assetNum': 'ST_1393137', 'siteId': 'DIST1'},{'assetNum': 'ST_1393138', 'siteId': 'DIST1'},{'assetNum': 'ST_1400501', 'siteId': 'DIST1'}]

# df = pmlib.api.get_asset_failure_history(assets=assets, data_items=None, df_id_column='id', df_timestamp_name='timestamp', async_mode=False)
# df.head()

2021-11-10T13:51:33.553 pmlib.api.init_environ INFO APM_ID=13f9a43a, APM_API_BASEURL=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc, APM_API_KEY=********
2021-11-10T13:51:33.554 pmlib.util.api_request INFO method=get, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/ds/tenant?instanceId=13f9a43a, headers={'apmapitoken': '********'}, timeout=300, ssl_verify=False, json=None, session=None, kwargs={}
2021-11-10T13:51:36.321 pmlib.util.api_request INFO resp.status_code=200, method=get, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/ds/tenant?instanceId=13f9a43a
2021-11-10T13:51:36.322 pmlib.api.init_environ DEBUG resp={
    "as_apikey": "********",
    "as_apitoken": "********",
    "as_id": null,
    "as_url": "https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com",
    "info": {
        "API_BASEURL": "https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com",
        "API_KEY": "********",
        "API

2021-11-10T13:51:37.471 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-05-18T00:00:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM5883489'}
2021-11-10T13:51:37.472 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-06-04T00:00:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM3051966'}
2021-11-10T13:51:37.472 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-06-30T00:00:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM4730587'}
2021-11-10T13:51:37.472 pmlib.api.get_asset_failure_history DEBUG response=<Response [200]>
2021-11-10T13:51:37.473 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-01-24T00:00:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM6769280'}
2021-11-10T13:51:37.473 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date'

id  timestamp    problemcode   faildate
0  ST_1393137-____-DIST2 2008-01-08  PUMPS/STOPPED 2008-01-08
1  ST_1393137-____-DIST2 2008-02-10  PUMPS/STOPPED 2008-02-10
2  ST_1393137-____-DIST2 2008-03-11  PUMPS/STOPPED 2008-03-11
3  ST_1393137-____-DIST2 2008-04-09  PUMPS/STOPPED 2008-04-09
4  ST_1393137-____-DIST2 2008-05-06  PUMPS/STOPPED 2008-05-06

Get the possible value of failure_mode

In [15]:
# df['problemcode'].unique()

array(['PUMPS/STOPPED'], dtype=object)

We will use 'PUMPS/LOWPRES' as failure_mode for the below example.

In [16]:
from sklearn.cluster import FeatureAgglomeration
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import NMF, PCA, FastICA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.ensemble import (
    AdaBoostRegressor,
    BaggingRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.exceptions import NotFittedError
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.kernel_approximation import (
    AdditiveChi2Sampler,
    Nystroem,
    RBFSampler,
    SkewedChi2Sampler,
)
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import (
    BayesianRidge,
    ElasticNet,
    HuberRegressor,
    Lasso,
    LassoLars,
    LinearRegression,
    OrthogonalMatchingPursuit,
    PassiveAggressiveRegressor,
    RANSACRegressor,
    Ridge,
    SGDRegressor,
    TheilSenRegressor,
)
from sklearn.manifold import Isomap, LocallyLinearEmbedding
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    PolynomialFeatures,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)
from sklearn.tree import DecisionTreeRegressor
from srom.auto.base_auto import SROMAutoPipeline
from srom.feature_engineering.model_based_feature_generator import (
    ModelbasedFeatureGenerator,
)
from srom.feature_selection.variance_based_feature_selection import (
    LowVarianceFeatureElimination,
)
from srom.pipeline.hyper_params.regression_fine_grid_for_bayesian import (
    PARAM_GRID as bayesian_paramgrid,
)
from srom.pipeline.hyper_params.regression_sample_grid_for_rbfopt import (
    PARAM_GRID as rbopt_paramgrid,
)
from srom.pipeline.srom_param_grid import SROMParamGrid
from srom.regression.data_partition_based_regression import PartitionRegressor
from srom.regression.gaussian_mixture_regressor import GaussianMixtureRegressor

from srom.utils.no_op import NoOp
from xgboost import XGBRegressor
from sklearn.preprocessing import KBinsDiscretizer, PowerTransformer




You can get the asset_group_id from the Maximo Health UI's "Predict grouping" page.

In [17]:
print(asset_group_id)
print(device_type_qualified_features)

1628
['Pump_DIST2_ST:VELOCITYX', 'Pump_DIST2_ST:VELOCITYY', 'Pump_DIST2_ST:VELOCITYZ']


In [18]:
from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline
from srom.utils.no_op import NoOp
group = pmlib.TimeToFailureAssetGroupPipeline(
                asset_group_id=asset_group_id, 
                model_pipeline={                 
                    'features':device_type_qualified_features,
                    'predictions': ['predicted_time_to_failure', 'predicted_failure_date'],
                    'features_resampled': {
                        iot_type: {
                            '${freqency}': '1D',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                    'failure_mode': 'PUMPS/STOPPED',                  
                    'aggregate_methods':['min','max'],
                    'aggregate_window_size': '3d',
                   # 'aggregate_type_for_prediction_interval': 'mean',
                    'smart_regression': True,
                    'execution_type': 'spark_node_random_search'               
                })


2021-11-10T13:51:52.318 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG in __init__ of AssetGroupPipeline, call api.init_environ()
2021-11-10T13:51:52.319 pmlib.api._get_db INFO db2
2021-11-10T13:51:52.319 pmlib.api._get_db INFO db2
2021-11-10T13:51:52.320 pmlib.api._get_db INFO DATABASE=BLUDB;HOSTNAME=c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc;PORT=50001;PROTOCOL=TCPIP;UID=admin;PWD=FJa6mTZkHf1mxeWW4YiP;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem
2021-11-10T13:51:54.803 iotfunctions.dbtables._store_table_exists DEBUG Table "MASWS_MAM"."KPI_MODEL_STORE" exists.
2021-11-10T13:51:54.804 iotfunctions.db.__init__ DEBUG Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2021-11-10T13:51:54.805 iotfunctions.db.__init__ DEBUG Found database type in os variable DB_TYPE: db2
2021-11-10T13:51:54.805 iotfunctions.db.__init__ DEBUG Found certificate filename in os

2021-11-10T13:51:57.480 pmlib.loader.AssetLoader._validate_data_items DEBUG all_entity_types should have both asset_group_id and entity type in IOTPlatform. For example, if you create PUMPSensor in IOTPlatform, PUMPSensor will be in the  all_entity_types
2021-11-10T13:51:57.481 pmlib.loader.AssetLoader._set_asset_device_mappings INFO input_asset_device_mappings=None, asset_device_mappings={}, entity_type_meta={}
2021-11-10T13:51:57.482 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG published_outputs={}
2021-11-10T13:51:57.482 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG prepared_published_outputs={}
2021-11-10T13:51:57.483 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG post_processing=[{'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_predicted_time_to_failure_pumps_stopped'}, 'input': {'source': 'predicted_time_to_failure_pumps_stopped'}}

The example above configured a pipeline for this model, accepting **```speed```, ```acc```,  and **```load```** of Watson IOT Platform device type **```ROBOT_TYPE```**. It also uses asset attributes **```installdate```** and **```faildate```** to extract the labels for training. The predicted output of the trained model instance is called **```predicted_time_to_failure```**.

By default, this model also generated daily aggregated prediction result taking output names in the form of **```daily_<predicted_failure_date_output_name>```**.

<a id="train-model-instance"></a>
## Train the Model Instance

With the model pipeline configured, now you can train the model instance:

In [19]:
# for AFM dataset , start_ts='2008-01-01',end_ts='2009-12-31'
# for OEM dataset, start_ts='2018-01-01',end_ts='2019-12-31'
#%time df = group.execute(start_ts='2008-01-01',end_ts='2009-12-31')
#%time df=group.execute(start_ts='2018-01-01',end_ts='2019-12-31')
%time df=group.execute()

2021-11-10T13:52:02.814 pmlib.api._validate_resampling DEBUG time_grain=day, agg_methods={'VELOCITYX': ['mean'], 'VELOCITYY': ['mean'], 'VELOCITYZ': ['mean']}, agg_outputs={'VELOCITYX': ['VELOCITYX'], 'VELOCITYY': ['VELOCITYY'], 'VELOCITYZ': ['VELOCITYZ']}
2021-11-10T13:52:02.814 pmlib.pipeline._ModelPipelineConfig.__init__ DEBUG features=['VELOCITYX', 'VELOCITYY', 'VELOCITYZ'], features_for_training=[], predictions=['predicted_time_to_failure', 'predicted_failure_date'], features_resampled={'Pump_DIST2_ST': ('day', {'VELOCITYX': ['mean'], 'VELOCITYY': ['mean'], 'VELOCITYZ': ['mean']}, {'VELOCITYX': ['VELOCITYX'], 'VELOCITYY': ['VELOCITYY'], 'VELOCITYZ': ['VELOCITYZ']})}, inputs=('Pump_DIST2_ST:VELOCITYX', 'Pump_DIST2_ST:VELOCITYY', 'Pump_DIST2_ST:VELOCITYZ'), renamed_inputs=('VELOCITYX', 'VELOCITYY', 'VELOCITYZ')
2021-11-10T13:52:02.815 pmlib.pipeline._ModelPipelineConfig.__init__ DEBUG kwargs={'failure_mode': 'PUMPS/STOPPED', 'aggregate_methods': ['min', 'max'], 'aggregate_window_siz

2021-11-10T13:52:02.821 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.execute DEBUG In the pipeline.py execute()  missing_value_threshold= 0.3
2021-11-10T13:52:02.822 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.execute DEBUG Skipping auto imputation
2021-11-10T13:52:02.822 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.__init__ DEBUG in MulticlassTimeToFailureEstimatorSrom __init()__ self.prediction_interval_low= prediction_interval_pumps_stopped_low
2021-11-10T13:52:02.823 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.__init__ DEBUG in MulticlassTimeToFailureEstimatorSrom __init()__ self.prediction_interval_high= prediction_interval_pumps_stopped_high
2021-11-10T13:52:02.823 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.__init__ DEBUG in MulticlassTimeToFailureEstimatorSrom __init()__ self.features_for_prediction= ['VELOCITYX', 'VELOCITYY', 'VELOCITYZ']
2021-11-10T13:52:02.823 pmlib.time_to_failure.MulticlassTimeToFailureEstimato

     site       asset devicetype deviceid
0   DIST2  ST_1393137       None     None
1   DIST2  ST_1393138       None     None
2   DIST2  ST_1400501       None     None
3   DIST2  ST_1400502       None     None
4   DIST2  ST_1400503       None     None
5   DIST2  ST_1400504       None     None
6   DIST2  ST_1400505       None     None
7   DIST2  ST_1400506       None     None
8   DIST2  ST_1400507       None     None
9   DIST2  ST_1400508       None     None
10  DIST2  ST_1400517       None     None
11  DIST2  ST_1400518       None     None
12  DIST2  ST_1400519       None     None
2021-11-10T13:52:05.735 pmlib.loader.AssetLoader._load_asset_device_mappings DEBUG 1. before df_pmi_mappings=shape=(13, 4), index={0: 'int64'}, columns={'site': 'O', 'asset': 'O', 'devicetype': 'O', 'deviceid': 'O'}, head(5)=
    site       asset devicetype deviceid
0  DIST2  ST_1393137       None     None
1  DIST2  ST_1393138       None     None
2  DIST2  ST_1400501       None     None
3  DIST2  ST_1400502  

2021-11-10T13:52:05.816 pmlib.loader.AssetLoader.execute DEBUG entity_type=Pump_DIST2_ST, data_items=['VELOCITYZ', 'VELOCITYX', 'VELOCITYY'], is_asset_data=False
2021-11-10T13:52:05.817 iotfunctions.db.http_request DEBUG URL: https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/kpi/v1/masws/entityType/Pump_DIST2_ST/engineInput
2021-11-10T13:52:05.902 iotfunctions.db.http_request DEBUG http request https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/kpi/v1/masws/entityType/Pump_DIST2_ST/engineInput successful. status 200
2021-11-10T13:52:05.905 iotfunctions.db.http_request DEBUG URL: https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/catalog/v1/masws/function?customFunctionsOnly=false
2021-11-10T13:52:06.040 iotfunctions.db.http_request DEBUG http request https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/catalog/v1/masws/function?customFunctionsOnly=false successful. status 200
2021-11-10T13:52:06.042 iotfunctions.db.load_catalog DEBUG Imported 

2021-11-10T13:52:06.212 iotfunctions.db.read_sql_query INFO exec_time_secs=0.05s sql=SELECT a.customtime, a.deviceid, AVG(DOUBLE((a."VELOCITYX"))) AS "VELOCITYX", AVG(DOUBLE((a."VELOCITYY"))) AS "VELOCITYY", AVG(DOUBLE((a."VELOCITYZ"))) AS "VELOCITYZ" FROM (SELECT date("MASWS_MAM"."IOT_PUMP_DIST2_ST".customtime) AS customtime, "MASWS_MAM"."IOT_PUMP_DIST2_ST".deviceid AS deviceid, "MASWS_MAM"."IOT_PUMP_DIST2_ST".velocityx AS "VELOCITYX", "MASWS_MAM"."IOT_PUMP_DIST2_ST".velocityy AS "VELOCITYY", "MASWS_MAM"."IOT_PUMP_DIST2_ST".velocityz AS "VELOCITYZ" FROM "MASWS_MAM"."IOT_PUMP_DIST2_ST") AS a GROUP BY a.customtime, a.deviceid
2021-11-10T13:52:06.217 iotfunctions.metadata.index_df DEBUG Indexed dataframe on id, customtime
2021-11-10T13:52:06.222 pmlib.api.get_entity_type_data DEBUG gate4: get sensor data from IOT_device_type table from IOTPlatform, df_raw=shape=(1196, 5), index={'id': 'O', 'customtime': '<M8[ns]'}, columns={'deviceid': 'O', 'VELOCITYX': 'float64', 'VELOCITYY': 'float64',

...

2021-11-10T13:59:57.670 srom.pipeline.execution_types.srom_spark_search.train_model DEBUG The total space of parameters is smaller than
                                    provided num_option_per_pipeline.
2021-11-10T13:59:57.671 srom.pipeline.execution_types.srom_spark_search.train_model DEBUG The total space of parameters is smaller than
                                    provided num_option_per_pipeline.
2021-11-10T13:59:57.671 srom.pipeline.execution_types.srom_spark_search.train_model DEBUG The total space of parameters is smaller than
                                    provided num_option_per_pipeline.
2021-11-10T13:59:57.672 srom.pipeline.execution_types.srom_spark_search.train_model DEBUG The total space of parameters is smaller than
                                    provided num_option_per_pipeline.
2021-11-10T13:59:57.672 srom.pipeline.execution_types.srom_spark_search.train_model DEBUG The total space of parameters is smaller than
                                    prov

2021-11-10T14:02:47.469 /opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pmlib/time_to_event_feature_eng.py.aggregate_scoring_data DEBUG From TSAggOps::aggregate_scoring_data: Columns for aggregation = ['VELOCITYZ', 'VELOCITYX', 'VELOCITYY']
2021-11-10T14:02:47.469 /opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pmlib/time_to_event_feature_eng.py.aggregate_scoring_data DEBUG From TSAggOps::aggregate_scoring_data: Aggregation function for columns = {'VELOCITYZ': ['min', 'max'], 'VELOCITYX': ['min', 'max'], 'VELOCITYY': ['min', 'max']}
2021-11-10T14:02:47.471 /opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pmlib/time_to_event_feature_eng.py.aggregate_scoring_data DEBUG From TSAggOps::aggregate_scoring_data: Reset the index of the incoming data frame. The columns are Index(['id', 'event_timestamp', 'VELOCITYX', 'VELOCITYY', 'VELOCITYZ'], dtype='object')
2021-11-10T14:02:47.472 /opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pmlib/time

2021-11-10T14:02:47.554 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.predict DEBUG in MulticlassTimeToFailureEstimatorSrom of predict function ,  input df : shape=(1196, 6), index={'id': 'O', 'event_timestamp': '<M8[ns]'}, columns={'VELOCITYZ__min': 'float64', 'VELOCITYZ__max': 'float64', 'VELOCITYX__min': 'float64', 'VELOCITYX__max': 'float64', 'VELOCITYY__min': 'float64', 'VELOCITYY__max': 'float64'}, head(5)=
                                       VELOCITYZ__min  VELOCITYZ__max  \
id                    event_timestamp                                   
ST_1393137-____-DIST2 2008-01-08             0.004033        0.004033   
                      2008-01-09             0.004033        0.011660   
                      2008-01-10             0.004033        0.019476   
                      2008-01-11             0.011660        0.026998   
                      2008-01-12             0.019476        0.034351   

                                       VELOCITYX__min  VEL

2021-11-10T14:03:04.857 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.process_prediction_result DEBUG prediction result again df_prediction=
                                       VELOCITYZ__min  VELOCITYZ__max  \
id                    event_timestamp                                   
ST_1393137-____-DIST2 2008-01-08             0.004033        0.004033   
                      2008-01-09             0.004033        0.011660   
                      2008-01-10             0.004033        0.019476   
                      2008-01-11             0.011660        0.026998   
                      2008-01-12             0.019476        0.034351   

                                       VELOCITYX__min  VELOCITYX__max  \
id                    event_timestamp                                   
ST_1393137-____-DIST2 2008-01-08             0.004048        0.004048   
                      2008-01-09             0.004048        0.011880   
                      2008-01-10          

2021-11-10T14:03:04.948 iotfunctions.pipeline._execute_stage DEBUG End of stage {{ MulticlassTimeToFailureEstimatorSrom }}, execution time = 653.078683 s
2021-11-10T14:03:04.949 iotfunctions.pipeline.validate_df WARNING Output dataframe is missing columns {'VELOCITYZ', 'VELOCITYX', 'VELOCITYY'} of type float64. Either the type has changed or column was dropped
2021-11-10T14:03:04.949 iotfunctions.pipeline.validate_df WARNING Output dataframe is missing columns {'faildate'} of type datetime64[ns]. Either the type has changed or column was dropped
2021-11-10T14:03:04.950 iotfunctions.pipeline.validate_df WARNING Output dataframe has no columns of type object. Type has changed or column was dropped.
2021-11-10T14:03:04.950 iotfunctions.pipeline._execute_stage DEBUG Function MulticlassTimeToFailureEstimatorSrom has no validate_df method. Skipping validation of the dataframe
2021-11-10T14:03:04.951 iotfunctions.db.get_table DEBUG Table name = APM_1628_AUTO_DIM , self.metadata = MetaData(bin

In [20]:
df

predicted_time_to_failure_pumps_stopped  \
id                    event_timestamp                                            
ST_1393137-____-DIST2 2008-01-08                                          29.0   
                      2008-01-09                                          28.0   
                      2008-01-10                                          28.0   
                      2008-01-11                                          26.0   
                      2008-01-12                                          25.0   
...                                                                        ...   
ST_1400504-____-DIST2 2021-11-06                                          14.0   
                      2021-11-07                                          17.0   
                      2021-11-08                                          17.0   
                      2021-11-09                                          16.0   
                      2021-11-10                                          15.0   

                                       prediction_interval_pumps_stopped_low  \
id                    event_timestamp                                          
ST_1393137-____-DIST2 2008-01-08                                        28.0   
                      2008-01-09                                        24.0   
                      2008-01-10                                        22.0   
                      2008-01-11                                        20.0   
                      2008-01-12                                        20.0   
...                                                                      ...   
ST_1400504-____-DIST2 2021-11-06                                        10.0   
                      2021-11-07                                        11.0   
                      2021-11-08                                        12.0   
                      2021-11-09                                        12.0   
                      2021-11-10                                        11.0   

                                       prediction_interval_pumps_stopped_high  \
id                    event_timestamp                                           
ST_1393137-____-DIST2 2008-01-08                                         31.0   
                      2008-01-09                                         30.0   
                      2008-01-10                                         30.0   
                      2008-01-11                                         29.0   
                      2008-01-12                                         27.0   
...                                                                       ...   
ST_1400504-____-DIST2 2021-11-06                                         19.0   
                      2021-11-07                                         21.0   
                      2021-11-08                                         21.0   
                      2021-11-09                                         21.0   
                      2021-11-10                                         21.0   

                                      predicted_failure_date_pumps_stopped  
id                    event_timestamp                                       
ST_1393137-____-DIST2 2008-01-08                                2008-02-06  
                      2008-01-09                                2008-02-06  
                      2008-01-10                                2008-02-07  
                      2008-01-11                                2008-02-06  
                      2008-01-12                                2008-02-06  
...                                                                    ...  
ST_1400504-____-DIST2 2021-11-06                                2021-11-20  
                      2021-11-07                                2021-11-24  
                      2021-11-08                                2021-11-25  
                      2021-11-

Once this method completes successfully, you'll have a trained model instance reday (for next step, see below) and also with the prediction results returned as a dataframe for verification.

<a id="register-trained-model-instance"></a>
## Register the Trained Model Instance


If the trained model instance looks good, you can register it to Maximo APM PMI:

In [21]:
group.register()

2021-11-10T14:03:23.396 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.register DEBUG target_pipeilne_class=pmlib.time_to_failure.TimeToFailureAssetGroupPipeline, url=None
2021-11-10T14:03:23.397 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.register DEBUG new model_template_id=TimeToFailureAssetGroupPipeline
2021-11-10T14:03:23.398 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.register DEBUG catalog_config={'name': 'TimeToFailureAssetGroupPipeline', 'description': 'TimeToFailureAssetGroupPipeline', 'moduleAndTargetName': 'pmlib.time_to_failure.TimeToFailureAssetGroupPipeline', 'url': 'https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/ds/13f9a43a/7n1okjdknc3e5nhptlmalj7vj5j1ak5ljppehd2t/lib/download?filename=pmlib', 'category': 'TRANSFORMER', 'tags': [], 'output': [{'name': 'names', 'description': 'Provide a list of output names to be generated from the pipeline.', 'dataType': 'ARRAY', 'jsonSchema': {'minItems': 1, '$schema': 'http://

2021-11-10T14:03:27.974 iotfunctions.metadata.register DEBUG found METRIC column format
2021-11-10T14:03:27.974 iotfunctions.metadata.register DEBUG found METRIC column event_timestamp
2021-11-10T14:03:27.974 iotfunctions.metadata.register DEBUG found METRIC column eventtype
2021-11-10T14:03:27.975 iotfunctions.metadata.register DEBUG found METRIC column logicalinterface_id
2021-11-10T14:03:27.975 iotfunctions.metadata.register DEBUG found METRIC column deviceid
2021-11-10T14:03:27.975 iotfunctions.metadata.register DEBUG found METRIC column devicetype
2021-11-10T14:03:27.975 iotfunctions.metadata.register DEBUG found METRIC column rcv_timestamp_utc
2021-11-10T14:03:27.976 iotfunctions.metadata.register DEBUG found METRIC column updated_utc
2021-11-10T14:03:27.976 iotfunctions.db.http_request DEBUG URL: https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/meta/v1/masws/entityType?createTables=true
2021-11-10T14:03:28.199 iotfunctions.db.http_request DEBUG http request https://

2021-11-10T14:04:08.922 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.save_model DEBUG saved apm/pmi/model/1628/MulticlassTimeToFailureEstimatorSrom/predicted_time_to_failure_pumps_stopped_1636552954_to_train.gz
2021-11-10T14:04:08.966 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.save_model DEBUG save to KPI_MODEL_STORE, pickle_dump=false
2021-11-10T14:04:08.966 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.save_model DEBUG in save_model() , before get_entity_type_id,  save to KPI_MODEL_STORE , db.model_store.entity_type_id=1217
2021-11-10T14:04:08.967 pmlib.time_to_failure.MulticlassTimeToFailureEstimatorSrom.save_model DEBUG In save_model(), save to KPI_MODEL_STORE , db.model_store.entity_type_id=1217
2021-11-10T14:04:08.974 iotfunctions.dbtables.store_model INFO Model apm/pmi/model/1628/MulticlassTimeToFailureEstimatorSrom/predicted_time_to_failure_pumps_stopped_1636552954_to_predict.gz of size 51199 bytes has been stored in table "MASWS_MAM".

2021-11-10T14:04:16.494 pmlib.util.api_request INFO resp.status_code=200, method=post, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/ds/7n1okjdknc3e5nhptlmalj7vj5j1ak5ljppehd2t/1628/model?instanceId=13f9a43a
2021-11-10T14:04:16.495 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.register DEBUG <Response [200]>
2021-11-10T14:04:16.496 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.register DEBUG model_instance_response={'modelInstanceId': '808D48AF-ADFA-4C18-9018-3537194184B1', 'message': 'Created', 'status': 0}
2021-11-10T14:04:16.497 iotfunctions.db.http_request DEBUG URL: https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/kpi/v1/masws/entityType/1628/engineInput
2021-11-10T14:04:16.684 iotfunctions.db.http_request DEBUG http request https://masws.api.monitor.ivt09rel86.ivt.suite.maximo.com/api/kpi/v1/masws/entityType/1628/engineInput successful. status 200
2021-11-10T14:04:16.686 pmlib.time_to_failure.TimeToFailureAssetGrou

2021-11-10T14:04:16.709 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._write INFO renamed_colsi={'predicted_time_to_failure_pumps_stopped|max': 'daily_predicted_time_to_failure_pumps_stopped', 'prediction_interval_pumps_stopped_low|max': 'daily_prediction_interval_pumps_stopped_low', 'prediction_interval_pumps_stopped_high|max': 'daily_prediction_interval_pumps_stopped_high', 'predicted_failure_date_pumps_stopped|max': 'daily_predicted_failure_date_pumps_stopped'}
2021-11-10T14:04:16.714 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._write DEBUG aggregation_df_grain=
shape=(1196, 4), index={'id': 'O', 'event_timestamp': '<M8[ns]'}, columns={'daily_predicted_time_to_failure_pumps_stopped': 'float64', 'daily_prediction_interval_pumps_stopped_low': 'float64', 'daily_prediction_interval_pumps_stopped_high': 'float64', 'daily_predicted_failure_date_pumps_stopped': '<M8[ns]'}, head(5)=
                                       daily_predicted_time_to_failure_pumps_stopped  \
id 

2021-11-10T14:04:16.744 iotfunctions.db.get_table DEBUG Table name = dm_1628 , self.metadata = MetaData(bind=Engine(db2+ibm_db://admin:***@c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc:50001/BLUDB;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem;))  
2021-11-10T14:04:16.760 pmlib.persist.PersistColumns.execute DEBUG clearing existing data of kpi=predicted_time_to_failure_pumps_stopped
2021-11-10T14:04:16.768 pmlib.persist.PersistColumns.execute DEBUG clearing existing data of kpi=prediction_interval_pumps_stopped_low
2021-11-10T14:04:16.772 pmlib.persist.PersistColumns.execute DEBUG clearing existing data of kpi=prediction_interval_pumps_stopped_high
2021-11-10T14:04:16.775 pmlib.persist.PersistColumns.execute DEBUG clearing existing data of kpi=predicted_failure_date_pumps_stopped
2021-11-10T14:04:16.782 pmlib.persist.PersistColumns.execute DEBUG in execute before call api._write_dataframe df=shape=(4784, 6), index={0: 'int64'}, columns={'entity

2021-11-10T14:04:17.133 pmlib.persist.PersistColumns.execute DEBUG target_table=dm_1628_daily, df_to_write=shape=(4784, 6), index={0: 'int64'}, columns={'entity_id': 'O', 'timestamp': '<M8[ns]', 'key': 'O', 'value_n': 'float64', 'value_t': '<M8[ns]', 'last_update': '<M8[ns]'}, head(5)=
               entity_id  timestamp  \
0  ST_1393137-____-DIST2 2008-01-08   
1  ST_1393137-____-DIST2 2008-01-08   
2  ST_1393137-____-DIST2 2008-01-08   
3  ST_1393137-____-DIST2 2008-01-08   
4  ST_1393137-____-DIST2 2008-01-09   

                                             key  value_n    value_t  \
0     daily_predicted_failure_date_pumps_stopped      NaN 2008-02-06   
1  daily_predicted_time_to_failure_pumps_stopped     29.0        NaT   
2   daily_prediction_interval_pumps_stopped_high     31.0        NaT   
3    daily_prediction_interval_pumps_stopped_low     28.0        NaT   
4     daily_predicted_failure_date_pumps_stopped      NaN 2008-02-06   

                 last_update  
0 2021-11-10 1

'808D48AF-ADFA-4C18-9018-3537194184B1'

In [22]:
# All time is UTC time, see following examples to set schedule to run every 5 minute, every hour, every day, every week, every month
# minute and second will be ignored. "2021-04-12 15:12:15" is same as "2021-04-12 15:00:00" in following examples
#group.enable(enabled=True, schedule={"starting_at": "2021-04-09 15:35:15", "every": "5min"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1H"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1D"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1W"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1M"})
group.enable(enabled=True,schedule={"starting_at": "05:00:01", "every": "5min"})

2021-11-10T14:04:17.527 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.enable DEBUG enabled=True, schedule={"starting_at": "05:00:01", "every": "5min"}, backtrack=null
2021-11-10T14:04:17.528 pmlib.util.api_request INFO method=put, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/assetgroup/1628/model/808D48AF-ADFA-4C18-9018-3537194184B1?instanceId=13f9a43a, headers={'apmapitoken': '********'}, timeout=300, ssl_verify=False, json={'enabled': True, 'schedule': {'starting_at': '05:00:01', 'every': '5min'}, 'backtrack': None}, session=None, kwargs={}
2021-11-10T14:05:56.101 pmlib.util.api_request INFO resp.status_code=200, method=put, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/assetgroup/1628/model/808D48AF-ADFA-4C18-9018-3537194184B1?instanceId=13f9a43a
2021-11-10T14:05:56.102 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.enable DEBUG <Response [200]>
2021-11-10T14:05:56.102 iotfunctions.db.get_table

<Response [200]>

In [ ]:
raise Exception("exit")

Sample TimeToFailureAssetGroupPipeline to compute the predicted time to failure using Cause in IBM MAXIMO.

In [25]:
from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline

group_cause = TimeToFailureAssetGroupPipeline(
            asset_group_id=asset_group_id, 
            model_pipeline={
                'features': device_type_qualified_features,
                'predictions': ['predicted_time_to_failure','predicted_failure_date'],
                'features_resampled': {
                        iot_type: {
                            '${freqency}': '1D',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                'failure_mode': 'PUMPS/STOPPED/MOTRFAIL',    
               
                'aggregate_methods':['min','max','std'],
                'aggregate_window_size': '3d',
                'aggregate_type_for_prediction_interval': 'mean',
                'smart_regression':True

            })

2021-10-26T21:20:41.345 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG in __init__ of AssetGroupPipeline, call api.init_environ()
2021-10-26T21:20:41.346 pmlib.api._get_db INFO db2
2021-10-26T21:20:41.346 pmlib.api._get_db INFO db2
2021-10-26T21:20:41.347 pmlib.api._get_db INFO DATABASE=BLUDB;HOSTNAME=c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc;PORT=50001;PROTOCOL=TCPIP;UID=admin;PWD=FJa6mTZkHf1mxeWW4YiP;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem
2021-10-26T21:20:41.898 iotfunctions.dbtables._store_table_exists DEBUG Table "MASWS_MAM"."KPI_MODEL_STORE" exists.
2021-10-26T21:20:41.899 iotfunctions.db.__init__ DEBUG Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2021-10-26T21:20:41.900 iotfunctions.db.__init__ DEBUG Found database type in os variable DB_TYPE: db2
2021-10-26T21:20:41.900 iotfunctions.db.__init__ DEBUG Found certificate filename in os

2021-10-26T21:20:43.224 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG published_outputs={}
2021-10-26T21:20:43.225 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG prepared_published_outputs={}
2021-10-26T21:20:43.226 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG post_processing=[{'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_predicted_time_to_failure_pumps_stopped_motrfail'}, 'input': {'source': 'predicted_time_to_failure_pumps_stopped_motrfail'}}, {'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_prediction_interval_pumps_stopped_motrfail_low'}, 'input': {'source': 'prediction_interval_pumps_stopped_motrfail_low'}}, {'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_prediction_interval_pumps_stopped_motrfail_high'}, 'input': {'source': 'prediction_i

Sample TimeToFailureAssetGroupPipeline to compute the predicted time to failure using Remedy in IBM MAXIMO.

In [26]:
from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline

group_remedy = TimeToFailureAssetGroupPipeline(
            asset_group_id=asset_group_id, 
            model_pipeline={
                'features': device_type_qualified_features,
                  'features_resampled': {
                        iot_type: {
                            '${freqency}': '1D',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                'predictions': ['predicted_time_to_failure','predicted_failure_date'],
                'failure_mode': 'PUMPS/STOPPED/MOTRFAIL/REPLACE',               
                'aggregate_methods':['min'],
                'aggregate_window_size': '3d',
                'aggregate_type_for_prediction_interval': 'mean',
                'smart_regression':True

            })

2021-10-26T21:20:52.977 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG in __init__ of AssetGroupPipeline, call api.init_environ()
2021-10-26T21:20:52.978 pmlib.api._get_db INFO db2
2021-10-26T21:20:52.979 pmlib.api._get_db INFO db2
2021-10-26T21:20:52.979 pmlib.api._get_db INFO DATABASE=BLUDB;HOSTNAME=c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc;PORT=50001;PROTOCOL=TCPIP;UID=admin;PWD=FJa6mTZkHf1mxeWW4YiP;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem
2021-10-26T21:20:53.595 iotfunctions.dbtables._store_table_exists DEBUG Table "MASWS_MAM"."KPI_MODEL_STORE" exists.
2021-10-26T21:20:53.599 iotfunctions.db.__init__ DEBUG Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2021-10-26T21:20:53.599 iotfunctions.db.__init__ DEBUG Found database type in os variable DB_TYPE: db2
2021-10-26T21:20:53.600 iotfunctions.db.__init__ DEBUG Found certificate filename in os

2021-10-26T21:20:55.690 pmlib.loader.AssetLoader._set_asset_device_mappings INFO input_asset_device_mappings=None, asset_device_mappings={}, entity_type_meta={}
2021-10-26T21:20:55.691 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG published_outputs={}
2021-10-26T21:20:55.692 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG prepared_published_outputs={}
2021-10-26T21:20:55.692 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG post_processing=[{'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_predicted_time_to_failure_pumps_stopped_motrfail_replace'}, 'input': {'source': 'predicted_time_to_failure_pumps_stopped_motrfail_replace'}}, {'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_prediction_interval_pumps_stopped_motrfail_replace_low'}, 'input': {'source': 'prediction_interval_pumps_stopped_motrfail_re

Once registration succeeds, you can see this newly trained model instance available for the asset group on IBM Maximo APM UI.

# How to overide the training stages

You can overide the training stages by providing your custom algorithms in the override_training_stages.

In [27]:
'''
from sklearn.cluster import FeatureAgglomeration
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import NMF, PCA, FastICA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.ensemble import (
    AdaBoostRegressor,
    BaggingRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.exceptions import NotFittedError
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.kernel_approximation import (
    AdditiveChi2Sampler,
    Nystroem,
    RBFSampler,
    SkewedChi2Sampler,
)
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import (
    BayesianRidge,
    ElasticNet,
    HuberRegressor,
    Lasso,
    LassoLars,
    LinearRegression,
    OrthogonalMatchingPursuit,
    PassiveAggressiveRegressor,
    RANSACRegressor,
    Ridge,
    SGDRegressor,
    TheilSenRegressor,
)
from sklearn.manifold import Isomap, LocallyLinearEmbedding
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    PolynomialFeatures,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)
from sklearn.tree import DecisionTreeRegressor
from srom.auto.base_auto import SROMAutoPipeline
from srom.feature_engineering.model_based_feature_generator import (
    ModelbasedFeatureGenerator,
)
from srom.feature_selection.variance_based_feature_selection import (
    LowVarianceFeatureElimination,
)
from srom.pipeline.hyper_params.regression_fine_grid_for_bayesian import (
    PARAM_GRID as bayesian_paramgrid,
)
from srom.pipeline.hyper_params.regression_sample_grid_for_rbfopt import (
    PARAM_GRID as rbopt_paramgrid,
)
from srom.pipeline.srom_param_grid import SROMParamGrid
from srom.regression.data_partition_based_regression import PartitionRegressor
from srom.regression.gaussian_mixture_regressor import GaussianMixtureRegressor

from srom.utils.no_op import NoOp
from xgboost import XGBRegressor
from sklearn.preprocessing import KBinsDiscretizer, PowerTransformer

'''

In [28]:
'''model_pipeline={
                    
                    'features':device_type_qualified_features,
                    'features_resampled': {
                        device_type: {
                            '${freqency}': '1D',
                            'VELOCITYX': {
                                
                                'mean': None,
                               
                            },
                            'VELOCITYY': {
                                
                                'mean': None,
                               
                            },
                            'VELOCITYZ': {
                                
                                'mean': None,
                               
                            },
                        },
                    },
                    
                    'predictions': ['predicted_time_to_failure', 'predicted_failure_date'],
                    'failure_mode': 'PUMPS/STOPPED',
                    
#                     'aggregate_methods':['min' 'max', 'median','std'],
                    'aggregate_methods':['min','max','std'],
                    'aggregate_window_size': '3d',  # for After market it is '3d'
                    'execution_type':'single_node_random_search',
#                     'aggregate_type_for_prediction_interval': 'mean',
                    'smart_regression':True,
                    'cv':3,
                    'override_training_stages': [
                        [
                            ("skiptransformation", NoOp()),
                            #("normalizer", Normalizer()),
                         ],
                        [
                            ('skipscaling', NoOp()), 
                            #('standardscaler', StandardScaler()),
                            #('robustscaler', RobustScaler()), 
                            #('minmaxscaling', MinMaxScaler())
                        ],
                        [
                             
                            #('XGBRegressor',XGBRegressor())  
                             ('RandomForestRegressor',RandomForestRegressor()) 
                           
                        ]
                    ]
                }
print(model_pipeline)'''

{'features': ['Pump_New_MULTICLASS:VELOCITYX', 'Pump_New_MULTICLASS:VELOCITYY', 'Pump_New_MULTICLASS:VELOCITYZ'], 'features_resampled': {'Pump_New_MULTICLASS': {'${freqency}': '1D', 'VELOCITYX': {'mean': None}, 'VELOCITYY': {'mean': None}, 'VELOCITYZ': {'mean': None}}}, 'predictions': ['predicted_time_to_failure', 'predicted_failure_date'], 'failure_mode': 'PUMPS/STOPPED', 'aggregate_methods': ['min', 'max', 'std'], 'aggregate_window_size': '3d', 'execution_type': 'single_node_random_search', 'smart_regression': True, 'cv': 3, 'override_training_stages': [[('skiptransformation', NoOp())], [('skipscaling', NoOp())], [('RandomForestRegressor', RandomForestRegressor())]]}


In [30]:
'''from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline
group = pmlib.TimeToFailureAssetGroupPipeline(
                asset_group_id=asset_group_id, 
                model_pipeline=model_pipeline)'''

2021-10-26T21:21:09.372 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG in __init__ of AssetGroupPipeline, call api.init_environ()
2021-10-26T21:21:09.373 pmlib.api._get_db INFO db2
2021-10-26T21:21:09.373 pmlib.api._get_db INFO db2
2021-10-26T21:21:09.374 pmlib.api._get_db INFO DATABASE=BLUDB;HOSTNAME=c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc;PORT=50001;PROTOCOL=TCPIP;UID=admin;PWD=FJa6mTZkHf1mxeWW4YiP;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem
2021-10-26T21:21:10.066 iotfunctions.dbtables._store_table_exists DEBUG Table "MASWS_MAM"."KPI_MODEL_STORE" exists.
2021-10-26T21:21:10.067 iotfunctions.db.__init__ DEBUG Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2021-10-26T21:21:10.067 iotfunctions.db.__init__ DEBUG Found database type in os variable DB_TYPE: db2
2021-10-26T21:21:10.068 iotfunctions.db.__init__ DEBUG Found certificate filename in os

2021-10-26T21:21:11.805 pmlib.loader.AssetLoader._set_asset_device_mappings INFO input_asset_device_mappings=None, asset_device_mappings={}, entity_type_meta={}
2021-10-26T21:21:11.805 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG published_outputs={}
2021-10-26T21:21:11.806 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline._prepare_published_outputs DEBUG prepared_published_outputs={}
2021-10-26T21:21:11.806 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.__init__ DEBUG post_processing=[{'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_predicted_time_to_failure_pumps_stopped'}, 'input': {'source': 'predicted_time_to_failure_pumps_stopped'}}, {'functionName': 'Maximum', 'enabled': True, 'granularity': 'Daily', 'output': {'name': 'daily_prediction_interval_pumps_stopped_low'}, 'input': {'source': 'prediction_interval_pumps_stopped_low'}}, {'functionName': 'Maximum', 'enabled': True, 'granular

ValueError: invalid model_pipeline.predictions having duplicated_names=['daily_predicted_time_to_failure_pumps_stopped', 'prediction_interval_pumps_stopped_high', 'daily_prediction_interval_pumps_stopped_high', 'prediction_interval_pumps_stopped_low', 'daily_prediction_interval_pumps_stopped_low', 'daily_predicted_failure_date_pumps_stopped', 'predicted_failure_date_pumps_stopped', 'predicted_time_to_failure_pumps_stopped'] with other_model_instances=['CE2E38CD-0FEB-4C01-A231-0D8CE667299B'] or with existing data items, please use different names

In [31]:
'''df=group.execute(start_ts='2018-01-01',end_ts='2019-12-31')'''

2021-10-26T21:21:23.556 pmlib.api._validate_resampling DEBUG time_grain=day, agg_methods={'VELOCITYX': ['mean'], 'VELOCITYY': ['mean'], 'VELOCITYZ': ['mean']}, agg_outputs={'VELOCITYX': ['VELOCITYX'], 'VELOCITYY': ['VELOCITYY'], 'VELOCITYZ': ['VELOCITYZ']}
2021-10-26T21:21:23.557 pmlib.pipeline._ModelPipelineConfig.__init__ DEBUG features=['VELOCITYX', 'VELOCITYY', 'VELOCITYZ'], features_for_training=[], predictions=['predicted_time_to_failure', 'predicted_failure_date'], features_resampled={'Pump_New_MULTICLASS': ('day', {'VELOCITYX': ['mean'], 'VELOCITYY': ['mean'], 'VELOCITYZ': ['mean']}, {'VELOCITYX': ['VELOCITYX'], 'VELOCITYY': ['VELOCITYY'], 'VELOCITYZ': ['VELOCITYZ']})}, inputs=('Pump_New_MULTICLASS:VELOCITYX', 'Pump_New_MULTICLASS:VELOCITYY', 'Pump_New_MULTICLASS:VELOCITYZ'), renamed_inputs=('VELOCITYX', 'VELOCITYY', 'VELOCITYZ')
2021-10-26T21:21:23.558 pmlib.pipeline._ModelPipelineConfig.__init__ DEBUG kwargs={'failure_mode': 'PUMPS/STOPPED', 'aggregate_methods': ['min', 'max'

2021-10-26T21:21:23.744 iotfunctions.util.__init__ DEBUG Starting trace
2021-10-26T21:21:23.745 iotfunctions.util.__init__ DEBUG Trace name: auto_trace_ASSET_CACHE_20211026212123
2021-10-26T21:21:23.745 iotfunctions.util.__init__ DEBUG auto_save None
2021-10-26T21:21:23.746 iotfunctions.util.categorize_args DEBUG categorizing arguments
2021-10-26T21:21:23.746 iotfunctions.db.get_table DEBUG Table name = APM_ASSET_CACHE , self.metadata = MetaData(bind=Engine(db2+ibm_db://admin:***@c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc:50001/BLUDB;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem;))  
2021-10-26T21:21:23.746 iotfunctions.metadata.__init__ DEBUG Initialized entity type 
EntityType:APM_ASSET_CACHE
Functions:
Granularities:
No schedules metadata
2021-10-26T21:21:23.747 pmlib.util.api_request INFO method=get, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/assetgroup/1580/devicemapping?instanceId=13f9a43a, head

2021-10-26T21:21:24.503 pmlib.loader.AssetLoader._load_asset_device_mappings DEBUG df_pmi_mappings=shape=(5, 5), index={0: 'int64'}, columns={'site': 'O', 'asset': 'O', 'devicetype': 'O', 'deviceid': 'O', 'assetfullid': 'O'}, head(5)=
      site            asset           devicetype         deviceid  \
0  BEDFORD  AP_NEWDEVICE001  Pump_New_MULTICLASS  AP_NEWDEVICE001   
1  BEDFORD  AP_NEWDEVICE003  Pump_New_MULTICLASS  AP_NEWDEVICE003   
2  BEDFORD  AP_NEWDEVICE005  Pump_New_MULTICLASS  AP_NEWDEVICE005   
3  BEDFORD  AP_NEWDEVICE007  Pump_New_MULTICLASS  AP_NEWDEVICE007   
4  BEDFORD  AP_NEWDEVICE009  Pump_New_MULTICLASS  AP_NEWDEVICE009   

                    assetfullid  
0  AP_NEWDEVICE001-____-BEDFORD  
1  AP_NEWDEVICE003-____-BEDFORD  
2  AP_NEWDEVICE005-____-BEDFORD  
3  AP_NEWDEVICE007-____-BEDFORD  
4  AP_NEWDEVICE009-____-BEDFORD  
ahi assetfullid_string AP_NEWDEVICE001-____-BEDFORD
<class 'str'>
ahi assetfullid_string AP_NEWDEVICE003-____-BEDFORD
<class 'str'>
ahi assetfulli

2021-10-26T21:21:24.527 iotfunctions.db.get_table DEBUG Table name = IOT_PUMP_NEW_MULTICLASS_CTG , self.metadata = MetaData(bind=Engine(db2+ibm_db://admin:***@c-db2wh-1628606058970938-db2u-engn-svc.cpd-meta-ops.svc:50001/BLUDB;SECURITY=SSL;SSLServerCertificate=/project_data/data_asset/db2_certificate.pem;))  
2021-10-26T21:21:24.531 iotfunctions.db.read_sql_query INFO The following sql statement is executed: SELECT a.customtime, a.deviceid, AVG(DOUBLE((a."VELOCITYX"))) AS "VELOCITYX", AVG(DOUBLE((a."VELOCITYY"))) AS "VELOCITYY", AVG(DOUBLE((a."VELOCITYZ"))) AS "VELOCITYZ" 
FROM (SELECT date("MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS".customtime) AS customtime, "MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS".deviceid AS deviceid, "MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS".velocityx AS "VELOCITYX", "MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS".velocityy AS "VELOCITYY", "MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS".velocityz AS "VELOCITYZ" 
FROM "MASWS_MAM"."IOT_PUMP_NEW_MULTICLASS" 
WHERE "MASWS_MAM"."IOT_PUMP_NEW_MULTICLAS

2021-10-26T21:21:27.429 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-05-18T12:12:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM2270137'}
2021-10-26T21:21:27.430 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-06-04T01:27:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM3233942'}
2021-10-26T21:21:27.430 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-06-30T02:40:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM1029277'}
2021-10-26T21:21:27.430 pmlib.api.get_asset_failure_history DEBUG response=<Response [200]>
2021-10-26T21:21:27.431 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date': '2008-05-07T04:49:00+00:00', 'classcode': 'PUMPS', 'problemcode': 'STOPPED', 'wonum': 'APM4528744'}
2021-10-26T21:21:27.431 pmlib.api.get_asset_failure_history DEBUG one_failure_history_record={'date'

2021-10-26T21:21:27.545 pmlib.api.get_asset_failure_history DEBUG in get_asset_failure_history data_item=['faildate', 'problemcode']
2021-10-26T21:21:27.545 pmlib.api.get_asset_failure_history DEBUG in get_asset_failure_history failure_columns={'faildate', 'problemcode'}
2021-10-26T21:21:27.549 pmlib.api.get_asset_failure_history DEBUG filtering asset time-series data: (start_ts=2018-01-01, end_ts=2019-12-31)
2021-10-26T21:21:27.552 pmlib.api.get_asset_failure_history DEBUG get_asset_failure_history before dropna df=shape=(0, 4), index={0: 'int64'}, columns={'asset_id': 'O', 'event_timestamp': '<M8[ns]', 'faildate': '<M8[ns]', 'problemcode': 'O'}, df=Empty DataFrame
2021-10-26T21:21:27.552 pmlib.api.get_asset_failure_history DEBUG get_asset_failure_history end failure_columns={'faildate', 'problemcode'}
2021-10-26T21:21:27.555 pmlib.api.get_asset_failure_history DEBUG AssetLoader end df=shape=(0, 4), index={0: 'int64'}, columns={'asset_id': 'O', 'event_timestamp': '<M8[ns]', 'faildate'

AttributeError: 'NoneType' object has no attribute 'update_stage_info'

In [33]:
'''group.register()'''

RuntimeError: load trained model by its model instance id before updating its registration

In [34]:
# All time is UTC time, see following examples to set schedule to run every 5 minute, every hour, every day, every week, every month
# minute and second will be ignored. "2021-04-12 15:12:15" is same as "2021-04-12 15:00:00" in following examples
#group.enable(enabled=True, schedule={"starting_at": "2021-04-09 15:35:15", "every": "5min"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1H"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1D"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1W"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1M"})
'''group.enable(enabled=True,schedule={"starting_at": "05:00:01", "every": "5min"})'''

2021-10-26T21:21:35.095 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.enable DEBUG enabled=True, schedule={"starting_at": "05:00:01", "every": "5min"}, backtrack=null
2021-10-26T21:21:35.096 pmlib.util.api_request INFO method=put, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/assetgroup/1580/model/CE2E38CD-0FEB-4C01-A231-0D8CE667299B?instanceId=13f9a43a, headers={'apmapitoken': '********'}, timeout=300, ssl_verify=False, json={'enabled': True, 'schedule': {'starting_at': '05:00:01', 'every': '5min'}, 'backtrack': None}, session=None, kwargs={}
2021-10-26T21:21:53.936 pmlib.util.api_request INFO resp.status_code=200, method=put, url=https://ivt09rel86-predict-api.mas-ivt09rel86-predict.svc/ibm/pmi/service/rest/assetgroup/1580/model/CE2E38CD-0FEB-4C01-A231-0D8CE667299B?instanceId=13f9a43a
2021-10-26T21:21:53.937 pmlib.time_to_failure.TimeToFailureAssetGroupPipeline.enable DEBUG <Response [200]>
2021-10-26T21:21:53.937 iotfunctions.db.get_table

<Response [200]>

##### Example of how to load the existing model instance 

In [ ]:
'''from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline
group = pmlib.TimeToFailureAssetGroupPipeline(
                asset_group_id=asset_group_id, 
                model_instance_id='0FEB51C6-A45D-464E-81B8-F7C7A34AFD64')'''

###### Example of scoring using the latest sensor data

In [ ]:
'''df=group.execute(start_ts='2021-06-01',end_ts='2021-06-26')'''

In [ ]:
'''group._write(df=df)'''

##### API of how to unregister the model so that you can r-train the model.

In [ ]:
'''from pmlib.time_to_failure import TimeToFailureAssetGroupPipeline
group = pmlib.TimeToFailureAssetGroupPipeline(
                asset_group_id=asset_group_id, 
                model_instance_id='BE809110-37DE-447A-9295-2ACF1D3BCBEE')'''

In [ ]:
'''group.unregister('0FEB51C6-A45D-464E-81B8-F7C7A34AFD64',force=True)'''

In [ ]:
store_json = json.dumps(store)
project.save_data("multi_execution.json", store_json, overwrite=True)